# Un premier graphe de compositeurs

In [35]:
from bs4 import BeautifulSoup
import requests
import re
import networkx as nx
import spacy

## Récupération des noms depuis Classic Cat

In [ ]:
url = requests.get("https://www.classiccat.net/composers.php")
page = BeautifulSoup(url.content, "html.parser")
li_compo = page.find_all("a", href = True)
compos = []
for lien in li_compo:
    text = lien.get_text()
    if "," in text:
        if len(compos)>0:
            if text[0] < compos[-1][0]:
                break
        compos.append(text)


In [ ]:
composers = [(compo.split(",")[1],compo.split(",")[0])  for compo in compos]

## Complétion des données à partir de Wikidata

In [ ]:
def find_info(soup, prop, dico, name):
    """Actualise le dictionnaire dico avec la première valeur de la propriété prop trouvée sur wikidata"""
    bloc = soup.find("div", id = prop)
    if bloc is None:
        res = None
    else:
        group = bloc.find("a",href=lambda href: href and href.startswith('/wiki/Q'))
        res = group['title'], group.get_text()
    dico[name]=res

In [ ]:
def complete_infos_all(name,dict_props):
    """Complète toutes les informations d'un compositeur à partir de son nom et de ses propriétés.
    Attention, le prédicat doit comporter un lien hypertexte"""
    dico = {"prénom" : name[0], "nom" : name[1]}
    url = f"https://www.wikidata.org/w/index.php?search={name[0]}+{name[1]}&title=Special%3ASearch&ns0=1&ns120=1"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    bloc = soup.find("div", class_ = "mw-search-result-heading")
    url = bloc.find("a")["href"]
    id = url[6:]
    dico["id"] = id
    page = requests.get("https://www.wikidata.org"+url)
    soup = BeautifulSoup(page.content, "html.parser")
    for nom,prop in dict_props.items():
        find_info(soup, prop, dico, nom)
    return dico

In [ ]:
li_props = {'nationalité' : 'P27',"sexe":"P21"}

In [73]:
complete_infos_all(("Johann-Sebastian","Bach"),li_props)

{'prénom': 'Johann-Sebastian',
 'nom': 'Bach',
 'id': 'Q1339',
 'nationalité': ('Q696651', 'Saxe-Eisenach'),
 'sexe': ('Q6581097', 'male'),
 'date de naissance': ('Q11148', 'The Guardian'),
 'date de décès': ('Q36578', 'Integrated Authority File')}